In [91]:
import pandas as pd
import numpy as np
import pickle 
from tqdm import tqdm 

def flatten_comprehension(matrix):
    #https://realpython.com/python-flatten-list/
    return [item for row in matrix for item in row]

# Load data
behaviors = pd.read_csv('./small_training_data/behaviors.tsv', delimiter='\t', header=None)
news = pd.read_csv('./small_training_data/news.tsv', delimiter='\t', header=None)

# Naming columns
behaviors.columns = ["impression_id", "user_id", "time", "history", "impressions"]
news.columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]


# Extracting clicked news from behaviors, this is a column of lists of the clicked news (tagget with 1) for each impression
behaviors['clicked_news'] = behaviors['impressions'].apply(lambda x: [imp.split('-')[0] for imp in x.split() if imp.split('-')[1] == '1'])

In [198]:
#define unique users and their read articles
unique_users_series = behaviors["user_id"].drop_duplicates()
unique_users_dict = {}  #user_id coupled to news_articles

#aggregate all read articles and couple it to the user in the unique_users_frame
i = 0
for user_id in tqdm(unique_users_series,ascii=True):
    user_specific_behavior = behaviors[behaviors["user_id"] == user_id]
    try:
        history_list = flatten_comprehension([elem.split(" ") for elem in user_specific_behavior["history"]])
    except:
        pass 
    clicked_list = flatten_comprehension([elem for elem in user_specific_behavior["clicked_news"]])
    all_read_articles = history_list + clicked_list
    unique_users_dict[user_id] = all_read_articles


with open('unique_users.pkl', 'wb') as file: #serialize so we don't have to do this inefficient code again
    pickle.dump(unique_users_dict,file)



100%|###################################################################################################################################################################| 50000/50000 [04:29<00:00, 185.77it/s]


In [197]:
with open('unique_users.pkl', 'rb') as file:
    test = pickle.load(file)
    print(test)

"""
make bag of word embedding table.
Then do cosine similarity (https://stats.stackexchange.com/questions/289400/quantify-the-similarity-of-bags-of-words)

X@X^T = matmul(X,X^T) (might be easy to compute)
then just select for each users the k users they are most similar.


"""
    

{'U13740': ['N55189', 'N42782', 'N34694', 'N45794', 'N18445', 'N63302', 'N10414', 'N19347', 'N31801', 'N55189', 'N42782', 'N34694', 'N45794', 'N18445', 'N63302', 'N10414', 'N19347', 'N31801', 'N55189', 'N42782', 'N34694', 'N45794', 'N18445', 'N63302', 'N10414', 'N19347', 'N31801', 'N55689', 'N28910', 'N58133'], 'U91836': ['N31739', 'N6072', 'N63045', 'N23979', 'N35656', 'N43353', 'N8129', 'N1569', 'N17686', 'N13008', 'N21623', 'N6233', 'N14340', 'N48031', 'N62285', 'N44383', 'N23061', 'N16290', 'N6244', 'N45099', 'N58715', 'N59049', 'N7023', 'N50528', 'N42704', 'N46082', 'N8275', 'N15710', 'N59026', 'N8429', 'N30867', 'N56514', 'N19709', 'N31402', 'N31741', 'N54889', 'N9798', 'N62612', 'N2663', 'N16617', 'N6087', 'N13231', 'N63317', 'N61388', 'N59359', 'N51163', 'N30698', 'N34567', 'N54225', 'N32852', 'N55833', 'N64467', 'N3142', 'N13912', 'N29802', 'N44462', 'N29948', 'N4486', 'N5398', 'N14761', 'N47020', 'N65112', 'N31699', 'N37159', 'N61101', 'N14761', 'N3433', 'N10438', 'N61355', '